# Import

In [68]:
import pandas as pd
import holidays
no_holidays = holidays.NO()

# Parametre & stier

In [44]:
DATA_PATH = "../data/raw_data/historical_flights.csv"
CUTOFF = "2024-01-01"

# Hjelpefunksjoner

In [45]:
def load_flights(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df = df[df["cancelled"] == 0].copy()
    for col in ["std", "sta", "atd", "ata"]:
        df[col] = pd.to_datetime(df[col], errors="coerce")
    return df

def handle_wrong_times(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['duration'] = df['sta'] - df['std']
    df = df[df['duration'] >= pd.Timedelta(0)].copy()
    df = df[df['duration'] <= pd.Timedelta(hours=10)].copy()
    return df

def make_intervals(df: pd.DataFrame, actual: bool = True) -> pd.DataFrame:
    if actual:
        dep = df.dropna(subset=["atd"]).copy()
        dep["start"] = dep["atd"] - pd.to_timedelta(15, "m")
        dep["end"]   = dep["atd"] + pd.to_timedelta(8, "m")
        dep["delay"] = (dep["atd"] - dep["std"]).dt.total_seconds() / 60

        arr = df.dropna(subset=["ata"]).copy()
        arr["start"] = arr["ata"] - pd.to_timedelta(16, "m")
        arr["end"]   = arr["ata"] + pd.to_timedelta(5, "m")
        arr["delay"] = (arr["ata"] - arr["sta"]).dt.total_seconds() / 60
    else:
        dep = df.dropna(subset=["std"]).copy()
        dep["start"] = dep["std"] - pd.to_timedelta(15, "m")
        dep["end"]   = dep["std"] + pd.to_timedelta(8, "m")
        dep["delay"] = 0

        arr = df.dropna(subset=["sta"]).copy()
        arr["start"] = arr["sta"] - pd.to_timedelta(16, "m")
        arr["end"]   = arr["sta"] + pd.to_timedelta(5, "m")
        arr["delay"] = 0  # <- FIX: delay på arrivals i scheduled

    dep["airport_group"] = dep["dep_airport_group"]
    dep["type"] = "departure"
    arr["airport_group"] = arr["arr_airport_group"]
    arr["type"] = "arrival"

    intervals = pd.concat([dep, arr], ignore_index=True)
    intervals = intervals.dropna(subset=["airport_group"])
    return intervals

def expand_to_hours(intervals: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for _, row in intervals.iterrows():
        hour_start = row["start"].floor("h")
        hour_end = row["end"].floor("h")
        hours = pd.date_range(hour_start, hour_end, freq="h")
        for h in hours:
            rows.append({**row, "hour": h})
    return pd.DataFrame(rows)

def hourly_overlap_group(group: pd.DataFrame) -> pd.DataFrame:
    hour = group["hour"].iloc[0]
    airport = group["airport_group"].iloc[0]
    events = []
    for _, row in group.iterrows():
        events.append((row["start"], +1))
        events.append((row["end"], -1))
    events.sort()
    active, overlap = 0, 0
    for _, change in events:
        active += change
        if active > 1:
            overlap = 1
            break
    return pd.DataFrame([{"airport_group": airport, "hour": hour, "target": overlap}])

def make_hourly_features(intervals_actual: pd.DataFrame) -> pd.DataFrame:
    intervals_actual = intervals_actual.copy()
    intervals_actual["duration_min"] = ((intervals_actual["sta"] - intervals_actual["std"]).dt.total_seconds() / 60)
    intervals_actual["airline"] = intervals_actual["flight_id"].str[:2]

    feats = intervals_actual.groupby(["airport_group", "hour"]).agg(
        flights_cnt     = ("flight_id", "count"),
        avg_duration    = ("duration_min", "mean"),
        max_duration    = ("duration_min", "max"),
        avg_delay       = ("delay", "mean"),
        max_delay       = ("delay", "max"),
        passenger_share = ("service_type", lambda x: (x == "J").mean()),
        cargo_share     = ("service_type", lambda x: (x == "P").mean()),
        charter_share   = ("service_type", lambda x: (x == "C").mean()),
        airline         = ("airline", lambda x: x.mode()[0])
    ).reset_index()


    feats["dow"]     = feats["hour"].dt.dayofweek
    feats["holiday"] = feats["hour"].apply(lambda x: x.date() in no_holidays)
    feats["month"]   = feats["hour"].dt.month
    feats["hournum"] = feats["hour"].dt.hour
    feats["weekend"] = (feats["dow"] >= 5).astype(int)

    feats["date"] = feats["hour"].dt.normalize()

    feats["daily_flights_cnt"] = feats.groupby(
        ["airport_group", "date"]
    )["flights_cnt"].transform("sum")

    feats = feats.sort_values(["airport_group", "hour"])
    feats["flights_cnt_prev"] = feats.groupby("airport_group")["flights_cnt"].shift(1)
    feats["flights_cnt_next"] = feats.groupby("airport_group")["flights_cnt"].shift(-1)
    feats[["flights_cnt_prev", "flights_cnt_next"]] = feats[["flights_cnt_prev", "flights_cnt_next"]].fillna(0).astype(int)
    return feats


# Last rådata

In [47]:
df_raw = load_flights(DATA_PATH)
df_raw

,flight_id,dep_airport,dep_airport_group,arr_airport,arr_airport_group,service_type,std,sta,cancelled,atd,ata
0,WF149,HOV,B,OSL,NaN,J,2018-01-02 16:40:00,2018-01-02 17:15:00,0,NaT,2018-01-02 18:53:00
1,WF722,OSL,NaN,MJF,D,J,2018-01-28 13:04:00,2018-01-28 14:50:00,0,NaT,NaT
2,WF188,FDE,A,OSL,NaN,J,2018-04-07 07:10:00,2018-04-07 08:10:00,0,NaT,2018-04-07 07:55:00
3,WF176,HOV,B,OSL,NaN,J,2018-04-07 11:00:00,2018-04-07 12:05:00,0,NaT,2018-04-07 12:00:00
4,WF148,HOV,B,OSL,NaN,J,2018-04-30 08:25:00,2018-04-30 09:26:00,0,NaT,2018-04-30 09:36:00
...,...,...,...,...,...,...,...,...,...,...,...
410437,WF153,SOG,A,HOV,B,J,2025-05-03 09:25:00,2025-05-03 10:00:00,0,2025-05-03 09:47:00,2025-05-03 10:19:00
410438,WF153,BGO,NaN,SOG,A,J,2025-05-03 08:35:00,2025-05-03 09:10:00,0,2025-05-03 08:29:00,2025-05-03 09:18:00
410439,WF158,OSL,NaN,HOV,B,J,2025-05-03 14:40:00,2025-05-03 15:50:00,0,2025-05-03 14:35:00,2025-05-03 15:39:00
410440,WF721,SSJ,D,TRD,NaN,J,2025-05-03 08:50:00,2025-05-03 09:45:00,0,2025-05-03 08:49:00,2025-05-03 09:39:00


# Rens og filter

In [48]:
df = handle_wrong_times(df_raw)
len(df), df.isna().mean().round(3).sort_values()

(399341,
 flight_id            0.000
 dep_airport          0.000
 arr_airport          0.000
 service_type         0.000
 std                  0.000
 sta                  0.000
 cancelled            0.000
 duration             0.000
 atd                  0.013
 ata                  0.015
 dep_airport_group    0.433
 arr_airport_group    0.434
 dtype: float64)

In [73]:
df

,flight_id,dep_airport,dep_airport_group,arr_airport,arr_airport_group,service_type,std,sta,cancelled,atd,ata,duration
0,WF149,HOV,B,OSL,NaN,J,2018-01-02 16:40:00,2018-01-02 17:15:00,0,NaT,2018-01-02 18:53:00,0 days 00:35:00
1,WF722,OSL,NaN,MJF,D,J,2018-01-28 13:04:00,2018-01-28 14:50:00,0,NaT,NaT,0 days 01:46:00
2,WF188,FDE,A,OSL,NaN,J,2018-04-07 07:10:00,2018-04-07 08:10:00,0,NaT,2018-04-07 07:55:00,0 days 01:00:00
3,WF176,HOV,B,OSL,NaN,J,2018-04-07 11:00:00,2018-04-07 12:05:00,0,NaT,2018-04-07 12:00:00,0 days 01:05:00
4,WF148,HOV,B,OSL,NaN,J,2018-04-30 08:25:00,2018-04-30 09:26:00,0,NaT,2018-04-30 09:36:00,0 days 01:01:00
...,...,...,...,...,...,...,...,...,...,...,...,...
410437,WF153,SOG,A,HOV,B,J,2025-05-03 09:25:00,2025-05-03 10:00:00,0,2025-05-03 09:47:00,2025-05-03 10:19:00,0 days 00:35:00
410438,WF153,BGO,NaN,SOG,A,J,2025-05-03 08:35:00,2025-05-03 09:10:00,0,2025-05-03 08:29:00,2025-05-03 09:18:00,0 days 00:35:00
410439,WF158,OSL,NaN,HOV,B,J,2025-05-03 14:40:00,2025-05-03 15:50:00,0,2025-05-03 14:35:00,2025-05-03 15:39:00,0 days 01:10:00
410440,WF721,SSJ,D,TRD,NaN,J,2025-05-03 08:50:00,2025-05-03 09:45:00,0,2025-05-03 08:49:00,2025-05-03 09:39:00,0 days 00:55:00


# Intervaller (actual) + hourly overlap (actual)

In [49]:
intervals_actual = make_intervals(df, actual=True)

intervals_actual_expanded = expand_to_hours(intervals_actual)
intervals_actual_expanded.head()

hourly_actual = (
    intervals_actual_expanded
    .groupby(["airport_group", "hour"], group_keys=False)
    .apply(hourly_overlap_group)
    .rename(columns={"target": "target_actual"})
)

hourly_actual.head()

/var/folders/fb/w8h543gs2z56q0kb6mdxsrgw0000gn/T/ipykernel_88790/3428016017.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(hourly_overlap_group)


,airport_group,hour,target_actual
0,A,2018-01-01 12:00:00,0
0,A,2018-01-01 13:00:00,1
0,A,2018-01-01 14:00:00,0
0,A,2018-01-01 16:00:00,1
0,A,2018-01-01 17:00:00,1


# Intervaller (scheduled) + hourly overlap (scheduled)

In [50]:
intervals_sched = make_intervals(df, actual=False)
intervals_sched_expanded = expand_to_hours(intervals_sched)

hourly_sched = (
    intervals_sched_expanded
    .groupby(["airport_group", "hour"], group_keys=False)
    .apply(hourly_overlap_group)
    .rename(columns={"target": "target_sched"})
)
hourly_sched.head()

/var/folders/fb/w8h543gs2z56q0kb6mdxsrgw0000gn/T/ipykernel_88790/1389039888.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(hourly_overlap_group)


,airport_group,hour,target_sched
0,A,2018-01-01 11:00:00,1
0,A,2018-01-01 12:00:00,1
0,A,2018-01-01 15:00:00,0
0,A,2018-01-01 16:00:00,1
0,A,2018-01-01 17:00:00,1


# Merge targets

In [51]:
hourly = hourly_actual.merge(hourly_sched, on=["airport_group","hour"], how="left")
hourly["target_sched"] = hourly["target_sched"].fillna(0).astype(int)
hourly["hour"] = pd.to_datetime(hourly["hour"])
hourly.head()

,airport_group,hour,target_actual,target_sched
0,A,2018-01-01 12:00:00,0,1
1,A,2018-01-01 13:00:00,1,0
2,A,2018-01-01 14:00:00,0,0
3,A,2018-01-01 16:00:00,1,1
4,A,2018-01-01 17:00:00,1,1


# Feature-agg (actual)

In [52]:
hourly_features = make_hourly_features(intervals_actual_expanded.copy())
hourly_features["hour"] = pd.to_datetime(hourly_features["hour"])
hourly_features.head()

,airport_group,hour,flights_cnt,avg_duration,max_duration,avg_delay,max_delay,passenger_share,cargo_share,charter_share,airline,dow,holiday,month,hournum,weekend,date,daily_flights_cnt,flights_cnt_prev,flights_cnt_next
0,A,2018-01-01 12:00:00,2,40.000000,50.0,8.000000,11.0,1.0,0.0,0.0,WF,0,True,1,12,0,2018-01-01,23,0,2
1,A,2018-01-01 13:00:00,2,60.000000,60.0,115.000000,116.0,1.0,0.0,0.0,WF,0,True,1,13,0,2018-01-01,23,2,1
2,A,2018-01-01 14:00:00,1,60.000000,60.0,114.000000,114.0,1.0,0.0,0.0,WF,0,True,1,14,0,2018-01-01,23,2,3
3,A,2018-01-01 16:00:00,3,61.666667,65.0,49.333333,61.0,1.0,0.0,0.0,WF,0,True,1,16,0,2018-01-01,23,1,4
4,A,2018-01-01 17:00:00,4,42.500000,60.0,35.000000,58.0,1.0,0.0,0.0,WF,0,True,1,17,0,2018-01-01,23,3,3


# Samle datasett + split

In [53]:
dataset = hourly.merge(hourly_features, on=["airport_group", "hour"], how="left").sort_values("hour")
train = dataset[dataset["hour"] < CUTOFF]
val   = dataset[dataset["hour"] >= CUTOFF]

train.shape, val.shape


((7, 22), (0, 22))

# lagre mellomfiler

In [54]:
train.to_csv('../data/processed_data/train.csv', index=False)
val.to_csv('../data/processed_data/val.csv', index=False)